In [1]:
# standard data science packages
import numpy as np
import pandas as pd

# imodels imports
from imodels.tree.rf_plus.rf_plus.rf_plus_models import \
    RandomForestPlusRegressor, RandomForestPlusClassifier
from imodels.tree.rf_plus.feature_importance.rfplus_explainer import \
    RFPlusMDI, AloRFPlusMDI

# functions for subgroup experiments
from subgroup_detection import *
from subgroup_experiment import *
import shap

# sklearn imports
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, LinearRegression, LogisticRegressionCV
from sklearn.metrics import roc_auc_score, average_precision_score, f1_score, \
    accuracy_score, r2_score, f1_score, log_loss, root_mean_squared_error

# pipeline imports
from subgroup import *

In [2]:
# set inputs
seed = 5
dataids = [361247, 361243, 361242, 361251, 361253, 361260, 361259, 361256, 361254, 361622]
dataid = 361617
clustertype = "hierarchical"
standardize = True

In [3]:
# get data
X, y = get_openml_data(dataid, standardize)

# split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.5,
                                                    random_state=seed)

print("Step 1")

# check if task is regression or classification
if len(np.unique(y)) == 2:
    task = 'classification'
else:
    task = 'regression'
    
# fit the prediction models
    rf, rf_plus_baseline, rf_plus_ridge, rf_plus_lasso, rf_plus_elastic = \
        fit_models(X_train, y_train, task)

print("Step 2")

# obtain shap feature importances
shap_explainer = shap.TreeExplainer(rf)
shap_test_values, shap_test_rankings = get_shap(X_test, shap_explainer,
                                                task)

print("Step 3")

# get lime feature importances
lime_test_values, lime_test_rankings = get_lime(X_test, rf, task)

print("Step 4")

# create list of lmdi variants
lmdi_variants = create_lmdi_variant_map()

# obtain lmdi feature importances
lmdi_explainers = get_lmdi_explainers(rf_plus_baseline, rf_plus_ridge,
                                          rf_plus_lasso, rf_plus_elastic,
                                          lmdi_variants)

print("Step 5")

# we don't actually want to use the training values, but for leaf averaging
# variants, we need to have the training data to compute the leaf averages
lfi_train_values, lfi_train_rankings = get_lmdi(X_train, y_train,
                                                lmdi_variants,
                                                lmdi_explainers)
lfi_test_values, lfi_test_rankings = get_lmdi(X_test, None,
                                                lmdi_variants,
                                                lmdi_explainers)

print("Step 6")

# add shap to the dictionaries
lfi_test_values["shap"] = shap_test_values
lfi_test_rankings["shap"] = shap_test_rankings

# add the raw data to the dictionaries as a baseline of comparison
lfi_test_values["rawdata"] = X_test

# add lime to the dictionaries
lfi_test_values["lime"] = lime_test_values
    
# get the clusterings - while we are not doing this on the training values,
# the get_train_clusters function still does what we want it to.
clusters = get_train_clusters(lfi_test_values, clustertype)

print("Step 7")

# for each cluster, assign half of the indices to the "fitting" set and
# the other half to the "evaluation" set
fitting_clusters = {}
evaluation_clusters = {}
for variant, nclust_map in clusters.items():
    fitting_nclust_to_c = {}
    evaluation_nclust_to_c = {}
    for nclust, cluster_map in nclust_map.items():
        fitting_c_to_idxs = {}
        evaluation_c_to_idxs = {}
        for c, idxs in cluster_map.items():
            if nclust == 3 and variant == "shap":
                print(f"Cluster {c} in variant {variant} has {len(idxs)} " + \
                    "observations.")
            if len(idxs) < 3:
                print(f"For {nclust} clusters, cluster #{c} in " + \
                    f"variant {variant} has fewer than 3 observations.")
                # # warning message that the cluster is too small
                # warnings.warn(f"For {nclust} clusters, cluster #{c} in " + \
                #     f"variant {variant} has fewer than 3 observations.",
                #     Warning)
                # continue
            # shuffle the indices and split them in half
            np.random.seed(1)
            np.random.shuffle(idxs)
            half = len(idxs) // 2
            fitting_c_to_idxs[c] = idxs[half:]
            evaluation_c_to_idxs[c] = idxs[:half]
        fitting_nclust_to_c[nclust] = fitting_c_to_idxs
        evaluation_nclust_to_c[nclust] = evaluation_c_to_idxs
    fitting_clusters[variant] = fitting_nclust_to_c
    evaluation_clusters[variant] = evaluation_nclust_to_c
    
print("Step 8")
    
# obtain dataframes X_fit, y_fit, X_eval, y_eval
# X_fit = []
# y_fit = []
# X_eval = []
# y_eval = []
# for variant, nclust_map in fitting_clusters.items():
#     for nclust, cluster_map in nclust_map.items():
#         for c, idxs in cluster_map.items():
#             X_fit.append(X_test[idxs])
#             y_fit.append(y_test[idxs])
# for variant, nclust_map in evaluation_clusters.items():
#     for nclust, cluster_map in nclust_map.items():
#         for c, idxs in cluster_map.items():
#             X_eval.append(X_test[idxs])
#             y_eval.append(y_test[idxs])
# X_fit = np.vstack(X_fit)
# y_fit = np.hstack(y_fit)
# X_eval = np.vstack(X_eval)
# y_eval = np.hstack(y_eval)

# print("X_fit shape", X_fit.shape)
# print("X_eval shape", X_eval.shape)

print("Step 9")
    
# compute the performance - we are using test data for both, not an error
metrics_to_scores = compute_performance(X_test, X_test, y_test, y_test,
                                        fitting_clusters,
                                        evaluation_clusters, task)


/tmp/ipykernel_1255945/1415906326.py:2: FutureWarning: Starting from Version 0.15.0 `download_splits` will default to ``False`` instead of ``True`` and be independent from `download_data`. To disable this message until version 0.15 explicitly set `download_splits` to a bool.
  X, y = get_openml_data(dataid, standardize)


Step 1


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    7.5s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    9.6s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  52 tasks      | elapsed:    1.3s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    2.1s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    1.5s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    4.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    4.7s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:   12.7s finished


Step 2
Step 3
Step 4
Step 5
Step 6
Step 7
Cluster 0 in variant shap has 196 observations.
Cluster 1 in variant shap has 130 observations.
Cluster 2 in variant shap has 58 observations.
Step 8
Step 9


/scratch/users/zachrewolinski/conda/envs/mdi/lib/python3.10/site-packages/sklearn/metrics/_regression.py:1187: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/scratch/users/zachrewolinski/conda/envs/mdi/lib/python3.10/site-packages/sklearn/metrics/_regression.py:1187: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/scratch/users/zachrewolinski/conda/envs/mdi/lib/python3.10/site-packages/sklearn/metrics/_regression.py:1187: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/scratch/users/zachrewolinski/conda/envs/mdi/lib/python3.10/site-packages/sklearn/model_selection/_search.py:1051: UserWarning: One or more of the test scores are non-finite: [nan nan nan]
  warnings.warn(
/scratch/users/zachrewolinski/conda/envs/mdi/lib/python3.10/site-packages/sklear

Cluster 0 in variant shap has train size 98
Cluster 0 in variant shap has RMSE 0.12602957420152133
model coef:
[ 2.33525366e-01  9.12450902e-02  1.84232726e-01 -3.67169284e-29
  0.00000000e+00  1.96345686e-02  1.77587422e-01  2.13156995e-02]
Cluster 1 in variant shap has train size 65
Cluster 1 in variant shap has RMSE 0.29213516919204807
model coef:
[ 0.02507316 -0.01844169  0.18070419 -0.10525567  0.          0.01895261
  0.28969734 -0.02109498]
Cluster 2 in variant shap has train size 29
Cluster 2 in variant shap has RMSE 0.13987568694155414
model coef:
[ 0.02483751 -0.02435783 -0.14754249  0.04750451  0.         -0.00197941
  0.33099578  0.07438016]


In [4]:
metrics_to_scores["rmse"]["lasso_l2_signed_nonnormed_noleafavg_rank"]

{2: 0.2417716092859805,
 3: 0.16758398898727805,
 4: 0.16874095091677,
 5: 0.16513650375694075,
 6: 0.17237257741486725,
 7: 0.17180984290037854,
 8: 0.17109069537905183,
 9: 0.17625155404750825,
 10: 0.1738707906178037}

In [5]:
# {2: 0.22644940353139587,
#  3: 0.19826779591905566,
#  4: 0.1953203953041628,
#  5: 0.18712289629518433,
#  6: 0.1799813913000766,
#  7: 0.18342879068896262,
#  8: 0.17688507989749783,
#  9: 0.18064867586887284,
#  10: 0.1718511382385029}

: 

: 